In [1]:
import cv2
import numpy as np

In [2]:
path_repo = 'C:/Users/fabri/Documents/GitHub/LudisAI/Squash/'
input_video = path_repo + 'squash-trim.avi'

In [6]:
vs = cv2.VideoCapture(input_video)
q_frames = int(vs.get(cv2.CAP_PROP_FRAME_COUNT))

frames_predict = np.random.choice(range(1, q_frames), size=int(q_frames*0.25), replace=False)

In [7]:
for img in frames_predict:
    vs.set(1, img)
    ret, frame = vs.read()
    if ret:
        
        #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #frame = cv2.resize(frame, (224, 224)).astype("float32")
        cv2.imshow('prueba', frame)

        #cv2.imshow("Output", output)
        key = cv2.waitKey(1) & 0xFF

        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break
        
vs.release()
cv2.destroyAllWindows()

In [ ]:
start_time = datetime.now()

# load the trained model and label binarizer from disk
print("[INFO] loading model and label binarizer...")
model = load_model(path_model + 'sports_classifier')
lb = pickle.loads(open(path_model + 'label_binarizer', "rb").read())

# initialize the image mean for mean subtraction along with the
# predictions queue
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")

# initialize the video stream, pointer to output video file, and
# frame dimensions
vs = cv2.VideoCapture(input_video)

q_frames = int(vs.get(cv2.CAP_PROP_FRAME_COUNT))
frames_predict = np.random.choice(range(1, q_frames), size=int(q_frames*0.25), replace=False)

pred_list = []

# loop over frames from the video file stream
print("[INFO] classifying video...")
for frame2read in frames_predict:
    # Set video frame to read
    vs.set(1, frame2read)
    
    # Read the next frame from the file
    (grabbed, frame) = vs.read()

    # Convert frame from BGR to RGB, resize the frame to a fixed 224x224, and then
    # perform mean subtraction
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224)).astype("float32")
    frame -= mean

    # Make predictions on the frame and then update the predictions list
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    pred_list.append(preds)

# release the file pointers
print("[INFO] cleaning up...")
vs.release()

print("[INFO] averaging predictions...")
results = np.array(pred_list).mean(axis=0)
i = np.argmax(results)
label = lb.classes_[i]

duration = datetime.now()-start_time
print('Duracion: ', duration)
print('El video es de ', label)